EOCanvas supports reading from or writing to an external storage location, such as an S3 bucket, EODATA, or the WEkEO Drive, by storing encrypted keys to the services. More information can be found in the documentation.

Here we show how to create S3 and Webdav (WEkEO Drive) keys.

Note that, in order to run this code, you have to setup your credentials as explained in _notebook.ipynb_

In [1]:
# Load all the necessary classes
from eocanvas.api import Key, WebDavKeyConfig, S3KeyConfig

# Set all the required parameters to configure a specific key
config = S3KeyConfig(
    access_key="*****",
    secret_key="*****",
    bucket="my-bucket",
    region="waw3-2",
    endpoint="https://s3.waw3-2.cloudferro.com",
)

# Note that the name must be unique. You might want to prefix your username.
key = Key(name="<your-username>-s3-key", config=config)

# Calling 'create' will download the public key from EOCanvas, encrypt the configuration and
# send it to the API.
key.create()

# Same thing for the WebDav case
config = WebDavKeyConfig(
    endpoint="https://wekeo-files.prod.wekeo2.eu/remote.php/dav/files/wso2_oidc-<your-username>",
    username="wso2_oidc-<your-username>",
    password="*****-*****-*****-*****-*****",
)
key = Key(name="<your-username>-wekeo-key", config=config)
key.create()

The Serverless API supports the following storage locations:

* S3 private buckets
* S3 public buckets
* EODATA
* WEkEO Drive

The following table clarifies if a storage location can be used for inputs, outputs or both:

| Storage location  | Suitable for inputs | Suitable for outputs |
| :---------------- | :-----------------: | :------------------: |
| S3 private bucket |        Yes          |          Yes         |
| S3 public bucket  |        Yes          |          No          |
| EODATA            |        Yes          |          No          |
| WEkEO Drive       |        No           |          Yes         |

Once a key has been added to the store, it can be used as an input source or an output destination, either by using it as an object or just by the name.

Here is an example using a WEkEO Drive key as input:

In [3]:
# As before, create all the required arguments to the process.
graph = Graph.from_uri("olci_binding.xml")
config = Config(key="img1", options=ConfigOption(uncompress=True, sub_path="xfdumanifest.xml"))
# Here we set an extra parameter 'keystore' and adjust the url with the path of the file
inputs = Input(keystore="<your-username>-wekeo-key", key="img1", url="/testing-inputs/S3A_OL_2_WFR____20220626T095133_20220626T095433_20220627T215353_0179_087_022_1980_MAR_O_NT_003.SEN3.zip")

You can then submit and run the process as usual:

In [4]:
process = SnapProcess(snap_graph=graph, eo_config=config, eo_input=inputs)
process.run()

To use a storage location as an external output, pass an 'output' parameter to the process:

In [ ]:
# 'output' can be either a Key instance or simply the name
process = SnapProcess(snap_graph=graph, eo_config=config, eo_input=inputs, output=Key(name="<your-username>-wekeo-key"))
process.run()

In this case, when the process is completed, the final products will be available at the selected storage and the download is not performed.